# Interactivity

In [7]:
# Import necessary packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from pathlib import Path
from ipywidgets import widgets, Layout
import folium
from IPython.display import clear_output

In [2]:
# Get cleaned gun violence master dataset 
directory = os.path.dirname(str(Path().resolve()) + '\\')
path = os.path.join(directory, 'data', 'gun-violence-data-cleaned.csv')
gunViolence = pd.read_csv(path)

In [12]:
button = widgets.Button(description = "Go!")
slider = widgets.IntSlider(min=0,max=5000,step=100,value=100,description='Number of Incidents:',layout=Layout(width='40%'),style={'description_width': 'initial'})
display(slider)
display(button)

def on_button_clicked(b):
    clear_output()
    display(slider)
    display(button)
    folium_map = folium.Map(location=[35, -97],
                            zoom_start=4,
                            tiles="Mapbox Bright")
    for i, row in gunViolence.head(slider.value).iterrows():
        marker = folium.CircleMarker(location=[row['latitude'], row['longitude']], radius = (row['n_killed'] + row['n_injured']))
        marker.add_to(folium_map)
    display(folium_map)

button.on_click(on_button_clicked)

IntSlider(value=1000, description='Number of Incidents:', layout=Layout(width='40%'), max=5000, step=100, styl…

Button(description='Go!', style=ButtonStyle())

In [17]:
# @widgets.interact
def selector(column='year', x=widgets.IntSlider(min=2009,max=2017,step=1,value=2017)):
    return gunViolence.loc[gunViolence[column] < x]

In [15]:
# Interactive map demonstration. 
# TODO: plot actual data and replace slider with date or other metric.
# @widgets.interact
def selector(x=widgets.FloatSlider(min=40.738,max=40.778,step=0.001,value=40.738)):
    folium_map = folium.Map(location=[40.738, -73.98],
                            zoom_start=13,
                            tiles="CartoDB dark_matter")
    marker = folium.CircleMarker(location=[x, -73.98], radius = 100)
    marker.add_to(folium_map)
#     for i, row in gunViolence.iterrows():
#         print(row['n_killed'])
#         marker = folium.CircleMarker(location=[row['latitude'], row['longitude']], radius = sum(row['n_killed'], row['n_injured']))
#         marker.add_to(folium_map)
    return folium_map

In [7]:
folium_map = folium.Map(location=[35, -97],
                            zoom_start=4,
                            tiles="Mapbox Bright")
for i, row in gunViolence.head(2000).iterrows():
    marker = folium.CircleMarker(location=[row['latitude'], row['longitude']], radius = (row['n_killed'] + row['n_injured']))
    marker.add_to(folium_map)
folium_map